# Деление на слоги

In [ ]:
from rusyll import rusyll

Пример работы библиотеки:

In [ ]:
rusyll.token_to_syllables("черепушидзе")

['че', 'ре', 'пу', 'ши', 'дзе']

# Ударения

In [ ]:
from russtress import Accent

Пример работы библиотеки: 

In [ ]:
accent = Accent()
text = 'Я вернулся в мой город, знакомый до слёз,'
accented_text = accent.put_stress(text)
accented_text

"Я верну'лся в мой го'род, знако'мый до слёз,"

# Ударение + Деление на слоги

In [ ]:
test = 'Я вернулся в мой город, знакомый до слёз,'

In [ ]:
li_unstressed = ['до', 'за', 'без', 'из', 'по']

In [ ]:
vowels = 'аеёиоуыэюя'

Ф-я добавляет графическое ударение там, где библиотечная функция не ставит (а именно в словах с одним гласным и на ё)

In [ ]:
def add_stress(text):
  res = []
  words = text.split()
  for word in words:
    w = word.strip(' .,-!?()\\"\\/*#№')
    if w not in li_unstressed and '\'' not in w:
      #print(True)
      letters = []
      for i in word:
        letters.append(i)
        if i.lower() in vowels:
          letters.append('\'')
      stress_fixed = ''.join(letters)
      res.append(stress_fixed)
    else:
      res.append(word)
  return " ".join(res)

In [ ]:
add_stress(accented_text)

"Я' верну'лся в мо'й го'род, знако'мый до слё'з,"

Ф-я совмещает деление на слоги и графическое ударение

In [ ]:
def rhythm(text):
  syl = '|'.join(rusyll.token_to_syllables(text))
  accented_previous = accent.put_stress(text)
  accented = add_stress(accented_previous)


  new_line = []
  max_len = max(len(text), len(syl), len(accented))
  idx_syl = 0
  idx_accented = 0
  idx_text = 0
  while idx_accented < max_len and idx_syl < max_len:
    #print(idx_accented, idx_syl, max_len)
    if text[idx_text] == accented[idx_accented]:
      idx_accented += 1
    else:
      new_line += accented[idx_accented]
      idx_accented += 2
    if text[idx_text] == syl[idx_syl]:
      idx_syl += 1
    else:
      new_line += syl[idx_syl]
      idx_syl += 2

    new_line += text[idx_text]
    idx_text += 1

  res = "".join(new_line)
  return res

In [ ]:
'|'.join(rusyll.token_to_syllables('Тот, кто любит цветы,'))

'То|т, кто| лю|би|т цве|ты,'

In [ ]:
rhythm(test)

"Я'| вер|ну'л|ся| в мо'й| го'|ро|д, зна|ко'|мый| до| слё'з,"

Ф-я превращает результат в список нулей и единиц, где 1 - ударный слог, 0 - безударный слог

In [ ]:
def rhythm_to_pattern(rhythmed):
  pattern = []
  data = rhythmed.split('|')
  for i in data:
    if '\'' in i:
      pattern.append(1)
    else:
      pattern.append(0)
  return pattern

In [ ]:
rhythm_to_pattern(rhythm(test))

[1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1]

## Корпус

In [ ]:
from google.colab import files
files.upload()

In [ ]:
import csv

In [ ]:
corpus = []
with open('poems_clean1.csv', encoding='utf-8', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter='|')
    for row in reader:
        corpus.append(row)

In [ ]:
corpus_clean = []
for i in corpus:
  if i:
    corpus_clean.append(i)

In [ ]:
def assess_rhythm(poem):
  lines = poem.split("\r\n")
  true_lines = []
  for i in lines:
    if i:
      true_lines.append(i)

  li_rhythm = []
  for line in true_lines:
    li_rhythm.append(rhythm_to_pattern(rhythm(line)))

  return li_rhythm

In [ ]:
sample1 = corpus_clean[-3][3]

In [ ]:
assess_rhythm(sample1)

[[1, 0, 0, 1, 0, 0, 1, 0],
 [1, 1, 0, 1, 0, 0, 1],
 [0, 0, 1, 0, 0, 1, 1, 1, 0],
 [1, 0, 1, 0, 0, 1, 0, 0, 1],
 [0, 1, 0, 0, 1, 0, 1],
 [1, 1, 0, 0, 1, 0, 1, 1, 0],
 [1, 0, 0, 1, 0, 0, 1, 1, 0],
 [1, 0, 1, 1, 1, 1, 1, 1, 1, 1],
 [0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0],
 [1, 1, 0, 1, 0, 0, 0, 1, 0],
 [0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0],
 [1, 1, 0, 0, 1, 0, 1, 1, 0, 1],
 [0, 1, 1, 1, 0, 0, 0, 0, 1, 0],
 [1, 0, 0, 0, 1, 1, 0, 1, 0, 0],
 [0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0],
 [0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
 [1, 0, 1, 0, 0, 1, 0, 1],
 [0, 1, 0, 1, 0, 1, 1, 0],
 [1, 1, 0, 1, 0, 0, 0, 1],
 [0, 1, 1, 1, 0, 1, 0, 1, 1, 0],
 [0, 1, 0, 0, 0, 1, 0, 1],
 [1, 1, 0, 1, 1, 0, 0, 1],
 [1, 0, 0, 1, 0, 0, 0, 0, 0, 1],
 [1, 0, 0, 1, 1, 0, 0, 1, 0, 0],
 [1, 0, 1, 1, 0, 1, 0, 1, 0],
 [1, 1, 0, 0, 0, 1, 0, 0, 1, 0],
 [1, 1, 0, 1, 1, 1, 1, 0, 0, 0],
 [0, 1, 0, 1, 1, 1, 0, 1, 0]]

**Ритм стихотворения характеризуется метром и размером.**

**Метр** - это повторение определенного соотношения ударного и безударного слогов. Например, если мы обозначим единицей ударные слоги, а нулем - безударные, то ямб можно записать ткак 01010101, дактиль - 100100100100.
**Размер** - это количество стоп (стопа - повторяющаяся группа ударных и безударных слогов в строке), то есть, проще говоря, размер - это длина строки.

Мы видим на сэмплах, что метр не определяется правильно нашим кодом (поскольку нет закономерности для каждой строки). Скорее всего это связано с тем, что в стихотворениями ударными могут являться не только те слоги, которые ударные "согласно словарю". Напротив, размер легко определить с помощью простого подсчета слогов в каждой строке. Это нормально, что длины чередуются (как в сэмплах 3 и 4, например) или даже имеют сложную схему длин, но стихотворение нельзя признать ритмичным, если длины строк произвольны (как в сэмплах 1 и 2). Поэтому в качестве показателя ритма мы возьмем отношение количества разных длин строк в стихе к его длине (количеству строк). Так, у сэмпла №3 всего две длины строк и всего 12 строк, поэтому показатель ритма будет равен 2/12 = 0.17. Поскольку при таком подходе мы получаем ситуацию, когда чем показатель меньше, тем лучше (в то время как другие параметры разметки ориентированы на противоположную ситуацию - "чем больше, тем лучше"), мы вычтем полученные показатели ритма из 1, чтобы инвертировать значения.

In [ ]:
def assess_syl_num(poem):
  lines = poem.split("\r\n")
  true_lines = []
  for i in lines:
    if i:
      true_lines.append(i)

  li_syl = []
  for line in true_lines:
    li_syl.append(len(rusyll.token_to_syllables(line)))

  len_diff_len_lines = 1 - len(list(set(li_syl))) / len(true_lines)
  len_diff = len(list(set(li_syl)))
  len_lines = len(true_lines)

  return [len_diff_len_lines, len_diff, len_lines]

In [ ]:
corpus_syl = []
corpus_syl.append(corpus_clean[0])
for i in corpus_clean[1:]:
  res = i.copy()
  syl_res = assess_syl_num(i[1])
  res.append(syl_res[0])
  res.append(syl_res[1])
  res.append(syl_res[2])
  corpus_syl.append(res)

In [ ]:
with open('syl_poems1.csv', 'w', encoding='utf-8') as f_w:
  csvwriter = csv.writer(f_w)
  csvwriter.writerow(['', 'text', 'name', 'difference in lines (divided by len)', 'difference in lines', 'len'])
  csvwriter.writerows(corpus_syl)